In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import sweetviz as sv
from data_exploratory import Profiles

data = Profiles(r"C:/Users/FrancescaConselvan/Dropbox/MODERATE/Enercoop//ENERCOOP_load_profiles.parquet.gzip",
                outputdir=r"C:/Users/FrancescaConselvan/Documents/MODERATE/GAN",
                load_profile_carrier="Enercoop")




KeyboardInterrupt: 